In [958]:
# class Word defines a word in dictionary; 
class Word(object):
    def __init__(self, WordName=None, WordIndex=None, WordFrequency=None):
        self.WordName = WordName
        self.WordIndex = WordIndex
        self.WordFrequency = WordFrequency


# class Element defines a word in a bug corpus or in a code corpus; 
class Element(object):
    def __init__(self, ElementName=None, ElementIndex=None, ElementFrequency=None):
        self.ElementName = ElementName
        self.ElementIndex = ElementIndex
        self.ElementFrequency = ElementFrequency
        self.ElementWeight = 0


# class Corpus defines a bug corpus or a code corpus; 
class Corpus(object):
    def __init__(self, FileName=None):
        self.FileName = FileName
        self.DocumentLength = 0
        self.Norm = 0
        self.rVSMScore = 0
        self.CorpusMembers = [] 

In [959]:
def generate_OneBugCorpus(FilePath, WordList):
    
    for i in range(len(WordList)):
        WordList[i].WordFrequency = 0
    
    with open(FilePath) as TextFile:
        ThisBugFile = TextFile.read().split(' ')    


    ThisBugCorpus = Corpus(FilePath) # this file delete word that is not in the word list; 

    for i in range(len(ThisBugFile)):
        flag = False
        for j in range(len(WordList)):
            if ThisBugFile[i] == WordList[j].WordName:
                flag = True
                WordList[j].WordFrequency = WordList[j].WordFrequency + 1
                break        
               

    for i in range(len(WordList)):
        if WordList[i].WordFrequency > 0:
            ThisBugCorpus.CorpusMembers.append(Element(WordList[i].WordName, 0, WordList[i].WordFrequency))            
        
    return ThisBugCorpus

In [960]:
def generate_OneCodeCorpus(ThisCodeFile, WordList):
    
    for i in range(len(WordList)):
        WordList[i].WordFrequency = 0
    
    ThisCodeCorpus = Corpus(ThisCodeFile[0])
    
    for i in range(len(ThisCodeFile)):
        flag = False
        for j in range(len(WordList)):
            if ThisCodeFile[i] == WordList[j].WordName:
                flag = True                    
                WordList[j].WordFrequency = WordList[j].WordFrequency + 1
                break
    
    for i in range(len(WordList)):
        if WordList[i].WordFrequency > 0:
            ThisCodeCorpus.CorpusMembers.append(Element(WordList[i].WordName, 0, WordList[i].WordFrequency))     
    
    return ThisCodeCorpus

In [961]:
def compute_BugCorpus_weight(CorpusNumber, MemberNumber):
    
    ThisWord = BugCorpuses[CorpusNumber].CorpusMembers[MemberNumber].ElementName
    # count ftd, the frequency of term t in document d;
    ftd = BugCorpuses[CorpusNumber].CorpusMembers[MemberNumber].ElementFrequency
#    print("ftd:", ftd)

    # #docs, total number of documents; 
    NumDocs = len(BugFiles) + len(CodeCorpuses);
#    print("#docs:", NumDocs)

    # count nt, the number of docuement that contains t;
    # count ntBug, the number of document that contains t in bug files; 
    ntBug = 0 
    for i in range(len(BugCorpuses)):    
        flag = False        
        for j in range(len(BugCorpuses[i].CorpusMembers)):
            if ThisWord == BugCorpuses[i].CorpusMembers[j].ElementName:
                flag = True
                break

        if flag == True:
            ntBug = ntBug + 1

#    print("ntBug:", ntBug)

    # count ntCode, the number of document that contains t in code files; 
    ntCode = 0
    for i in range(len(CodeCorpuses)):
        flag = False
        for j in range(len(CodeCorpuses[i].CorpusMembers)):
            if ThisWord == CodeCorpuses[i].CorpusMembers[j].ElementName:
                flag = True
                break

        if flag == True:
                ntCode = ntCode + 1

#    print("ntCode:", ntCode)

    nt = ntBug + ntCode
#    print("nt:", nt)

    weight = (math.log(ftd)+1) * math.log(NumDocs/nt)
#    print(weight)
    
    return weight

In [962]:
def compute_CodeCorpus_weight(CorpusNumber, MemberNumber):
    
    ThisWord = CodeCorpuses[CorpusNumber].CorpusMembers[MemberNumber].ElementName
    # count ftd, the frequency of term t in document d;
    ftd = CodeCorpuses[CorpusNumber].CorpusMembers[MemberNumber].ElementFrequency
#    print("ftd:", ftd)

    # #docs, total number of documents; 
    NumDocs = len(BugFiles) + len(CodeCorpuses);
#    print("#docs:", NumDocs)

    # count nt, the number of docuement that contains t;
    # count ntBug, the number of document that contains t in bug files; 
    ntBug = 0 
    for i in range(len(BugCorpuses)):    
        flag = False        
        for j in range(len(BugCorpuses[i].CorpusMembers)):
            if ThisWord == BugCorpuses[i].CorpusMembers[j].ElementName:
                flag = True
                break

        if flag == True:
            ntBug = ntBug + 1

#    print("ntBug:", ntBug)

    # count ntCode, the number of document that contains t in code files; 
    ntCode = 0
    for i in range(len(CodeCorpuses)):
        flag = False
        for j in range(len(CodeCorpuses[i].CorpusMembers)):
            if ThisWord == CodeCorpuses[i].CorpusMembers[j].ElementName:
                flag = True
                break

        if flag == True:
                ntCode = ntCode + 1

#    print("ntCode:", ntCode)

    nt = ntBug + ntCode
#    print("nt:", nt)

    weight = (math.log(ftd)+1) * math.log(NumDocs/nt)
#    print(weight)
    
    return weight

In [963]:
def compute_BugCorpus_norm(CorpusNumber):
    norm = 0
    for i in range(len(BugCorpuses[CorpusNumber].CorpusMembers)):
        norm = norm + BugCorpuses[CorpusNumber].CorpusMembers[i].ElementWeight**2
        
    norm = norm**0.5
    
    return norm

In [964]:
def compute_CodeCorpus_norm(CorpusNumber):
    norm = 0
    for i in range(len(CodeCorpuses[CorpusNumber].CorpusMembers)):
        norm = norm + CodeCorpuses[CorpusNumber].CorpusMembers[i].ElementWeight**2
        
    norm = norm**0.5
    
    return norm

In [965]:
def compute_intersection(BugCorpusNumber, CodeCorpusNumber):
    intersection = 0
    for i in range(len(BugCorpuses[BugCorpusNumber].CorpusMembers)):
        for j in range(len(CodeCorpuses[CodeCorpusNumber].CorpusMembers)):
            if BugCorpuses[BugCorpusNumber].CorpusMembers[i].ElementName == CodeCorpuses[CodeCorpusNumber].CorpusMembers[j].ElementName:
                intersection += BugCorpuses[BugCorpusNumber].CorpusMembers[i].ElementWeight * CodeCorpuses[CodeCorpusNumber].CorpusMembers[j].ElementWeight
                break
    
    return intersection    

In [966]:
def compute_rVSMScore(BugCorpusNumber, CodeCorpusNumber):
    intersection = compute_intersection(BugCorpusNumber, CodeCorpusNumber)
    rVSM_score = intersection / (BugCorpuses[BugCorpusNumber].Norm * CodeCorpuses[CodeCorpusNumber].Norm)
    
    
    return rVSM_score

In [967]:
# this process intends to save all file names in an array; 

import os

FolderPath = 'C:\\Users\\take_\\OneDrive\\liming\\where\\BugLocator\\tmpMine\\'
path = FolderPath + 'BugCorpus\\'

BugFiles = []
# r=root, d=directories, f = bug files
for r, d, f in os.walk(path):
    for file in f:
        if '.txt' in file:
            BugFiles.append(os.path.join(r, file))

print(len(BugFiles))

3


In [972]:

## ----------------------------------- read word list from Wordlist.txt --------------------------------------------
with open(FolderPath + "WordList.txt") as TextFile:
    WordlistContent = [line.split() for line in TextFile]

WordList = []
    
for i in range(len(WordlistContent)):    
    WordList.append(Word(WordlistContent[i][0], WordlistContent[i][1], 0))
    
# test whether WordList is correct or not;     
print(WordList[0].WordName)
print(WordList[1].WordName)
    
    
## ----------------------------------- read all bug files ------------------------------------------------    
#ThisBugCorpus = generate_OneBugCorpus(BugFiles[0], WordList)
#print(ThisBugCorpus.FileName)
#print(ThisBugCorpus.CorpusMembers)

BugCorpuses = []
for i in range(len(BugFiles)):
    ThisBugCorpus = generate_OneBugCorpus(BugFiles[i], WordList)
    BugCorpuses.append(ThisBugCorpus)

# test result of all bug corpuses;     
# print(BugCorpuses[0])
for i in range(1):
    print(BugCorpuses[i].FileName)
    for j in range(len(BugCorpuses[i].CorpusMembers)):
        print(BugCorpuses[i].CorpusMembers[j].ElementName, end = ',')
        print(BugCorpuses[i].CorpusMembers[j].ElementFrequency, end = ',')
    print()

## ----------------------------------- read all code files ------------------------------------------------
with open(FolderPath + "CodeCorpus.txt") as TextFile:
    CodeList = TextFile.read().split()   

#print(len(CodeList))
CodeFiles = []
ThisCodeFile = []

for i in range(len(CodeList)):
    if CodeList[i][-5:] == ".java":        
        CodeFiles.append(ThisCodeFile)
        ThisCodeFile = []
        ThisCodeFile.append(CodeList[i])
    else:
        ThisCodeFile.append(CodeList[i])

CodeFiles.append(ThisCodeFile)
CodeFiles.pop(0) # delete the first element;         

#print(len(CodeFiles))
#print(CodeFiles[0])

#ThisCodeCorpus = generate_OneCodeCorpus(CodeFiles[0], WordList)
#print(ThisCodeCorpus)

# generate code corpuses from code files;
CodeCorpuses = []
for i in range(len(CodeFiles)):
    ThisCodeCorpus = generate_OneCodeCorpus(CodeFiles[i], WordList)
    CodeCorpuses.append(ThisCodeCorpus)
    
# compute code length to each code corpuses; 
for i in range(len(CodeCorpuses)):
    CodeCorpuses[i].DocumentLength = len(CodeFiles[i])

# find maximum and minimum document length of code files;
MinDocumentLength = CodeCorpuses[0].DocumentLength
MaxDocumentLength = CodeCorpuses[0].DocumentLength
for i in range(len(CodeCorpuses)):
    if CodeCorpuses[i].DocumentLength < MinDocumentLength:
        MinDocumentLength = CodeCorpuses[i].DocumentLength
    if CodeCorpuses[i].DocumentLength > MaxDocumentLength:
        MaxDocumentLength = CodeCorpuses[i].DocumentLength
    
#print(MinDocumentLength)
#print(MaxDocumentLength)
    
# test all code corpuses generated; 
print(CodeCorpuses[0].FileName)
#print(CodeCorpuses[0].CorpusMembers)
for i in range(len(CodeCorpuses[0].CorpusMembers)):
    print(CodeCorpuses[0].CorpusMembers[i].ElementName, end = ',')
    print(CodeCorpuses[0].CorpusMembers[i].ElementFrequency, end = ',')
print()

aaa
bbb
C:\Users\take_\OneDrive\liming\where\BugLocator\tmpMine\BugCorpus\14654.txt
aaa,3,bbb,2,ccc,1,
3
8
org.eclipse.swt.accessibility.ACC.java
aaa,3,xxx,1,


In [969]:
import math

#compute weight for each element in bug corpuses
for i in range(len(BugCorpuses)):
    for j in range(len(BugCorpuses[i].CorpusMembers)):
        BugCorpuses[i].CorpusMembers[j].ElementWeight= compute_BugCorpus_weight(i, j)
    
# compute norm for each document in bug corpuses;
for i in range(len(BugCorpuses)):
    BugCorpuses[i].Norm = compute_BugCorpus_norm(i)

# test result of element weight;
for i in range(2):
    print(BugCorpuses[i].FileName)
    print(BugCorpuses[i].Norm)
    for j in range(len(BugCorpuses[i].CorpusMembers)):
        print("[", end = '')
        print(BugCorpuses[i].CorpusMembers[j].ElementName, end = ',')
        print(BugCorpuses[i].CorpusMembers[j].ElementFrequency, end = ',')
        print(BugCorpuses[i].CorpusMembers[j].ElementWeight, end = '')
        print("]", end = ',')
    print()

C:\Users\take_\OneDrive\liming\where\BugLocator\tmpMine\BugCorpus\14654.txt
1.0479273243962555
[aaa,3,0.38262225957689383],[bbb,2,0.686512104608772],[ccc,1,0.6931471805599453],
C:\Users\take_\OneDrive\liming\where\BugLocator\tmpMine\BugCorpus\58185.txt
0.8234659178637195
[aaa,1,0.1823215567939546],[bbb,1,0.4054651081081644],[ccc,1,0.6931471805599453],


In [970]:
#compute weight for each element in the code corpuses; 
for i in range(len(CodeCorpuses)):
    for j in range(len(CodeCorpuses[i].CorpusMembers)):
        CodeCorpuses[i].CorpusMembers[j].ElementWeight= compute_CodeCorpus_weight(i, j)

# compute norm for each document in code corpuses;
for i in range(len(CodeCorpuses)):
    CodeCorpuses[i].Norm = compute_CodeCorpus_norm(i)        
        
# test result of element weight;
for i in range(2):
    print(CodeCorpuses[i].FileName)
    print(CodeCorpuses[i].Norm)
    for j in range(len(CodeCorpuses[i].CorpusMembers)):
        print("[", end = '')
        print(CodeCorpuses[i].CorpusMembers[j].ElementName, end = ',')
        print(CodeCorpuses[i].CorpusMembers[j].ElementFrequency, end = ',')
        print(CodeCorpuses[i].CorpusMembers[j].ElementWeight, end = '')
        print("]", end = ',')
    print()

org.eclipse.swt.accessibility.ACC.java
1.1633351857209124
[aaa,3,0.38262225957689383],[xxx,1,1.0986122886681098],
org.eclipse.swt.accessibility.Accessible.java
3.9360533876001695
[aaa,3,0.38262225957689383],[xxx,1,1.0986122886681098],[zzz,3,3.7602084404594462],


In [971]:
# compute rVSMScore; 
for i in range(len(CodeCorpuses)):
    CodeCorpuses[i].rVSMScore = compute_rVSMScore(0, i)

for i in range(len(CodeCorpuses)):
    print(CodeCorpuses[i].rVSMScore)

0.12008933387429925
0.03549345848963112
0.14459266009106922
